# Daily Fill Aggressiveness

Measure how aggressively weekly orders are getting filled each trading day.

Uses `wolfpack/targets.csv` and computes:
- `incremental_fill_pct`: percent of the weekly order filled *on that day* (same-day observed)
- `cumulative_fill_pct`: percent of the weekly order filled *by that day* (same-day observed)
- `incremental_fill_pct_settled_t1`: same metric but using next-day `actual_w` as a T+1 settled proxy
- `cumulative_fill_pct_settled_t1`: cumulative T+1 settled proxy

Plots are shown by:
- calendar weekday (Mon-Fri)
- scale day index (0..N)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


In [ ]:
df_targets = read_csv_from_store('wolfpack/targets.csv')

if df_targets is None:
    raise ValueError('targets.csv is required. Run a backtest with target-state logging first.')

df_targets['date'] = pd.to_datetime(df_targets['date'])
numeric_cols = ['start_w', 'weekly_target_w', 'scheduled_fraction', 'scheduled_w', 'actual_w', 'scale_day']
for col in numeric_cols:
    if col in df_targets.columns:
        df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)

print(f'target rows: {len(df_targets):,}')
display(df_targets.head())


In [ ]:
df = df_targets.copy()

df = df.sort_values(['symbol', 'date'])

# Weekly order size and remaining amount (same-day observed)
df['weekly_order_abs'] = (df['weekly_target_w'] - df['start_w']).abs()
df['remaining_abs'] = (df['weekly_target_w'] - df['actual_w']).abs()
df['remaining_abs'] = np.minimum(df['remaining_abs'], df['weekly_order_abs'])

# T+1 settled proxy: use next trading day's actual weight for the same symbol.
df['actual_w_settled_t1'] = df.groupby('symbol')['actual_w'].shift(-1)
df['actual_w_settled_t1'] = df['actual_w_settled_t1'].fillna(df['actual_w'])

# Keep canonical sort for week/day calculations.
df = df.sort_values(['week_id', 'symbol', 'date'])

# Same-day incremental/cumulative fill.
df['prior_remaining_abs'] = df.groupby(['week_id', 'symbol'])['remaining_abs'].shift(1)
df['prior_remaining_abs'] = df['prior_remaining_abs'].fillna(df['weekly_order_abs'])
df['incremental_fill_abs'] = (df['prior_remaining_abs'] - df['remaining_abs']).clip(lower=0)

df['incremental_fill_pct'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    df['incremental_fill_abs'] / df['weekly_order_abs'],
    0.0
)

df['cumulative_fill_pct'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    (df['weekly_order_abs'] - df['remaining_abs']) / df['weekly_order_abs'],
    1.0
)

# T+1 settled incremental/cumulative fill.
df['remaining_abs_settled_t1'] = (df['weekly_target_w'] - df['actual_w_settled_t1']).abs()
df['remaining_abs_settled_t1'] = np.minimum(df['remaining_abs_settled_t1'], df['weekly_order_abs'])

df['prior_remaining_abs_settled_t1'] = df.groupby(['week_id', 'symbol'])['remaining_abs_settled_t1'].shift(1)
df['prior_remaining_abs_settled_t1'] = df['prior_remaining_abs_settled_t1'].fillna(df['weekly_order_abs'])
df['incremental_fill_abs_settled_t1'] = (
    df['prior_remaining_abs_settled_t1'] - df['remaining_abs_settled_t1']
).clip(lower=0)

df['incremental_fill_pct_settled_t1'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    df['incremental_fill_abs_settled_t1'] / df['weekly_order_abs'],
    0.0
)

df['cumulative_fill_pct_settled_t1'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    (df['weekly_order_abs'] - df['remaining_abs_settled_t1']) / df['weekly_order_abs'],
    1.0
)

# Clip to [0,1] for stable plotting.
for col in [
    'incremental_fill_pct',
    'cumulative_fill_pct',
    'incremental_fill_pct_settled_t1',
    'cumulative_fill_pct_settled_t1'
]:
    df[col] = df[col].clip(0, 1)

df['weekday'] = df['date'].dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)

df = df[df['weekly_order_abs'] > 1e-10].copy()

display(
    df[[
        'date', 'week_id', 'symbol', 'scale_day',
        'incremental_fill_pct', 'cumulative_fill_pct',
        'incremental_fill_pct_settled_t1', 'cumulative_fill_pct_settled_t1'
    ]].head(15)
)


In [ ]:
# Portfolio-level daily fill aggressiveness (weighted by order size)
by_date = (
    df.groupby('date', as_index=False)
      .agg(
          incremental_fill_abs=('incremental_fill_abs', 'sum'),
          incremental_fill_abs_settled_t1=('incremental_fill_abs_settled_t1', 'sum'),
          weekly_order_abs=('weekly_order_abs', 'sum')
      )
)
by_date['incremental_fill_pct'] = np.where(
    by_date['weekly_order_abs'] > 1e-10,
    by_date['incremental_fill_abs'] / by_date['weekly_order_abs'],
    0.0
)
by_date['incremental_fill_pct_settled_t1'] = np.where(
    by_date['weekly_order_abs'] > 1e-10,
    by_date['incremental_fill_abs_settled_t1'] / by_date['weekly_order_abs'],
    0.0
)
by_date['weekday'] = pd.Categorical(by_date['date'].dt.day_name(), categories=weekday_order, ordered=True)

weekday_weighted = (
    by_date.groupby('weekday', as_index=False)
           .agg(
               avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
               avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean')
           )
)

weekday_symbol_mean = (
    df.groupby('weekday', as_index=False)
      .agg(
          avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
          avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean'),
          med_incremental_fill_pct=('incremental_fill_pct', 'median'),
          avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
          avg_cumulative_fill_pct_settled_t1=('cumulative_fill_pct_settled_t1', 'mean')
      )
)

display(weekday_weighted)
display(weekday_symbol_mean)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

weekday_plot = weekday_weighted.copy()
weekday_plot['same_day_pct'] = 100 * weekday_plot['avg_incremental_fill_pct']
weekday_plot['settled_t1_pct'] = 100 * weekday_plot['avg_incremental_fill_pct_settled_t1']
weekday_long = weekday_plot.melt(
    id_vars='weekday',
    value_vars=['same_day_pct', 'settled_t1_pct'],
    var_name='series',
    value_name='value'
)
weekday_long['series'] = weekday_long['series'].map({
    'same_day_pct': 'Same-day observed',
    'settled_t1_pct': 'T+1 settled proxy'
})

sns.barplot(
    data=weekday_long,
    x='weekday',
    y='value',
    hue='series',
    ax=axes[0]
)
axes[0].set_title('Weighted Average % Filled Per Day (Calendar Weekday)')
axes[0].set_ylabel('% of weekly order filled on that day')
axes[0].set_xlabel('Weekday')
axes[0].grid(axis='y', alpha=0.3)

sns.lineplot(
    data=weekday_symbol_mean,
    x='weekday',
    y=100 * weekday_symbol_mean['avg_cumulative_fill_pct'],
    marker='o',
    color='#d62728',
    label='Same-day observed',
    ax=axes[1]
)
sns.lineplot(
    data=weekday_symbol_mean,
    x='weekday',
    y=100 * weekday_symbol_mean['avg_cumulative_fill_pct_settled_t1'],
    marker='o',
    color='#1f77b4',
    label='T+1 settled proxy',
    ax=axes[1]
)
axes[1].set_title('Average Cumulative Fill % By Weekday')
axes[1].set_ylabel('Cumulative fill %')
axes[1].set_xlabel('Weekday')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
scale_day_profile = (
    df.groupby('scale_day', as_index=False)
      .agg(
          avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
          avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean'),
          med_incremental_fill_pct=('incremental_fill_pct', 'median'),
          avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
          avg_cumulative_fill_pct_settled_t1=('cumulative_fill_pct_settled_t1', 'mean')
      )
      .sort_values('scale_day')
)

display(scale_day_profile)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

x = scale_day_profile['scale_day'].to_numpy()
width = 0.38
axes[0].bar(x - width/2, 100 * scale_day_profile['avg_incremental_fill_pct'], width=width, color='#2ca02c', label='Same-day observed')
axes[0].bar(x + width/2, 100 * scale_day_profile['avg_incremental_fill_pct_settled_t1'], width=width, color='#1f77b4', label='T+1 settled proxy')
axes[0].set_title('Average % Filled Per Scale Day')
axes[0].set_xlabel('Scale day')
axes[0].set_ylabel('% of weekly order filled that day')
axes[0].set_xticks(x)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

axes[1].plot(scale_day_profile['scale_day'], 100 * scale_day_profile['avg_cumulative_fill_pct'], marker='o', color='#9467bd', label='Same-day observed')
axes[1].plot(scale_day_profile['scale_day'], 100 * scale_day_profile['avg_cumulative_fill_pct_settled_t1'], marker='o', color='#ff7f0e', label='T+1 settled proxy')
axes[1].set_title('Average Cumulative Fill % by Scale Day')
axes[1].set_xlabel('Scale day')
axes[1].set_ylabel('Cumulative fill %')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Optional: symbol-level ranking of aggressiveness on day 0 and day 1
day_early = df[df['scale_day'].isin([0, 1])]
symbol_early = (
    day_early.groupby('symbol', as_index=False)
             .agg(avg_fill_day0_1=('incremental_fill_pct', 'mean'),
                  observations=('incremental_fill_pct', 'count'))
             .sort_values('avg_fill_day0_1', ascending=False)
)

print('Most aggressive early fills (day 0-1):')
display(symbol_early.head(15))
